In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier


#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

//anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
# preprocessing 

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submission=pd.read_csv('sample_submission.csv')

prepros=train.copy()
prepros.drop(['gender'],inplace=True,axis=1)
prepros=prepros.append(test, ignore_index=True)   # dataframe containing test and train


In [3]:
 train['gender'] = LabelEncoder().fit_transform(train['gender'])

In [4]:
ProductList_processed=[]
ProductList_processed_bask_no=[]
ProductList_processed_no=[]
for i in range(prepros.shape[0]):
    ProductList_processed.append((prepros['ProductList'][i].strip().replace(';','').replace('/',' ')))
    ProductList_processed_no.append(len(prepros['ProductList'][i].strip().replace(';','').replace('/',' ')))
    ProductList_processed_bask_no.append(len(list(prepros['ProductList'][i].split(';'))))

In [6]:
prepros['ProductList_processed']=ProductList_processed
prepros['ProductList_processed_bask_no']=ProductList_processed_bask_no
prepros['ProductList_processed_no']=ProductList_processed_no

In [7]:
corpus=ProductList_processed
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
cols=vectorizer.get_feature_names()

In [8]:
X_m=X.toarray()
prod2vec=pd.DataFrame(data=X_m,
          index=np.array(range(15000)),
          columns=cols)

In [9]:
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)

svd.fit(X)

X_tr=svd.fit_transform(X)

#X_embedded=TSNE(n_components=2).fit_transform(X_tr)

In [10]:
#plt.scatter(X_embedded[:,0],X_embedded[:,1])

In [11]:
prod2vec_red=pd.DataFrame(data=X_tr,
          index=np.array(range(15000)),
          columns=np.array(range(50)))

In [12]:
prepros=pd.concat([prepros, prod2vec_red], axis=1, sort=False)
prepros_p2v=prepros.copy()

In [13]:
prepros_p2v.drop(['ProductList','ProductList_processed'],inplace=True,axis=1)

In [14]:
prepros_p2v['startTime']=pd.to_datetime(prepros_p2v['startTime'])
prepros_p2v['endTime']=pd.to_datetime(prepros_p2v['endTime'])

In [15]:
prepros_p2v['duration']=(prepros_p2v['endTime']-prepros_p2v['startTime']).astype('timedelta64[m]')

In [16]:
#prepros_p2v['duration_mod']=prepros_p2v[(prepros_p2v['duration']>0) & (prepros_p2v['duration']<100)]['duration']

In [17]:
#prepros_p2v['duration_mod']=prepros_p2v['duration_mod'].fillna(m)

In [18]:
#prepros_p2v['duration_mod']
#prepros_p2v.drop(['duration'],inplace=True,axis=1)

In [19]:
prepros_p2v['hour']=prepros_p2v['startTime'].dt.hour

In [20]:
prepros_p2v['weekday']=(prepros_p2v['startTime'].dt.weekday_name).astype('category')

In [21]:
prepros_p2v['day']=prepros_p2v['startTime'].dt.day

In [22]:
prepros_p2v['month']=prepros_p2v['startTime'].dt.month

In [23]:
prepros_p2v['weekday'] = LabelEncoder().fit_transform(prepros_p2v['weekday'])

In [24]:
prepros_p2v_f=prepros_p2v.drop(['startTime','endTime'], axis=1)

In [25]:
####training and test split####

pro_train=prepros_p2v_f[:train.shape[0]]

In [26]:
pro_test=prepros_p2v_f[train.shape[0]:]

In [27]:
pro_test=pro_test.drop(['session_id'],axis=1)
pro_train=pro_train.drop(['session_id'],axis=1)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(pro_train, 
                                                    train['gender'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [29]:
# # xgb=XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=6,
#  min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
#  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

# xgb.fit(pro_train, train['gender'])

clf = XGBClassifier( learning_rate =0.1, n_estimators=182, max_depth=7,
 min_child_weight=5, gamma=0.1, subsample=0.9, colsample_bytree=0.9,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

eval_set  = [(X_train, y_train),(X_test, y_test)]

clf.fit(X_train, y_train,
            eval_set=eval_set, eval_metric='error',verbose=0)

pred = clf.predict_proba(X_test)[:,1]
accuracy = accuracy_score(y_test, np.round(pred))
print( "SCORE:", accuracy)

SCORE: 0.8717460317460317


In [30]:
def prep_sub(clf):
    pred = clf.predict(pro_test)
    Pred=pd.DataFrame(pred)
    mapping = {0: 'female', 1: 'male'}
    Pred=Pred.replace({0: mapping, 1: mapping})
    submission['gender']=Pred[0]
    submission.to_csv('submission.csv',index=False)
    
    
prep_sub(clf)